# Chains

* [1. Chains and Why They Are Used](#chains)
    * [1.1. LLMChain](#LLMChain) 
    * [1.2. Parsers](#parsers) 
    * [1.3. Conversational Chain (Memory)](#memory) 
    * [1.4. Sequential Chain](#seq)
    * [1.5. Debug](#debug) 
    * [1.6. Custom](#custom) 
* [2. YouTube Video Summarizer](#video_summarizer)
* [3. Creating Voice Assistant](#voice_assistant)
* [4. Code Comprehension - Twitter Algorithm](#code_understanding)
* [5. Recommendation Engine for Songs](#recommendation)
* [6. Self-Critique Chain](#critique)
* [7. Additional Resources](#resources)

In [1]:
import os
from keys import OPENAI_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

<hr>
<a class="anchor" id="chains">
    
## 1. Chains and the Reasons to Use them
    
</a>

Chains are responsible for creating an end-to-end pipeline to use LLMs. Chains join the model, prompt, memory, parsing output, debugging capability. They also provide an easy-to-use interface. 

A chain will:
1) receive the user’s query as an input;
2) process the LLM’s response;
3) return the output to the user.

It is possible to design a custom pipeline by inheriting the `Chain` class. 

<hr>
<a class="anchor" id="LLMChain">
    
### 1.1. LLMChain
    
</a>

There are several methods to use chains:
- **`__ call __`** pass an input directly to the object while initializing it; will return the input variable and the model’s response under the text key;
- **`.apply()`** pass multiple inputs at once and receive a list for each input;
- **`.generate()`** returns an instance of `LLMResult`, which provides more information;
- **`.predict()`** pass multiple (or single) inputs for a single prompt;
- **`.run()`** the same as .predict (they can be used interchangeably).

In [2]:
from langchain import PromptTemplate, OpenAI, LLMChain

In [3]:
prompt_template = "What is a word to replace the following: {word}?"


llm = OpenAI(model_name="text-davinci-003", temperature=0)

llm_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate.from_template(prompt_template)
)

In [4]:
# Passing input directly to object while initializing it >> __call__ 
llm_chain("artificial") 

{'word': 'artificial', 'text': '\n\nSynthetic'}

In [5]:
# Passing multiple inputs  >> .apply() 
input_list = [
    {"word": "artificial"},
    {"word": "intelligence"},
    {"word": "robot"}
]

llm_chain.apply(input_list)

[{'text': '\n\nSynthetic'}, {'text': '\n\nWisdom'}, {'text': '\n\nAutomaton'}]

In [6]:
# Return an instance of LLMResult with more information >> .generate()
llm_chain.generate(input_list)

LLMResult(generations=[[Generation(text='\n\nSynthetic', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text='\n\nWisdom', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text='\n\nAutomaton', generation_info={'finish_reason': 'stop', 'logprobs': None})]], llm_output={'token_usage': {'total_tokens': 46, 'prompt_tokens': 33, 'completion_tokens': 13}, 'model_name': 'text-davinci-003'}, run=[RunInfo(run_id=UUID('12354ad4-428d-4577-afed-9bf2f543fdd4')), RunInfo(run_id=UUID('846124ec-28fe-42f9-9bf3-818e7f79afa7')), RunInfo(run_id=UUID('fe91d0d7-e93e-41b0-a123-b1ba6bbe585e'))])

In [7]:
# Pass both the word and the context
prompt_template = "Looking at the context of '{context}'. What is an appropriate word to replace the following: {word}?"

llm_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate(template=prompt_template, input_variables=["word", "context"]))

llm_chain.predict(word="present", context="object")

'\n\ngift'

In [8]:
# alternative to .predict()
llm_chain.run(word="present", context="object")

'\n\ngift'

In [9]:
llm_chain.predict(word="present", context="time") # or llm_chain.run(word="present", context="time")

'\n\nNow'

**Note**: To format the output we can use either parsers (see example below and refer to [notebook section](06.%20Prompting.ipynb#outputs)) or we can directly pass a prompt as a string to a Chain and initialize it using the .from_string() function as follows:
`LLMChain.from_string(llm=llm, template=template)`

<hr>
<a class="anchor" id="parsers">
    
### 1.2. Parsers
    
</a>

In [13]:
from langchain.output_parsers import CommaSeparatedListOutputParser

output_parser = CommaSeparatedListOutputParser()
template = """List all possible words as substitute for 'artificial' as comma separated."""

llm_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate(template=template, output_parser=output_parser, input_variables=[]),
    output_parser=output_parser)

llm_chain.predict()

['Synthetic',
 'Manufactured',
 'Imitation',
 'Fabricated',
 'Fake',
 'Mechanical',
 'Computerized',
 'Automated',
 'Simulated',
 'Artificial Intelligence',
 'Constructed',
 'Programmed',
 'Processed',
 'Algorithmic.']

<hr>
<a class="anchor" id="memory">
    
### 1.3. Conversational Chain (Memory)
    
</a>

LangChain provides a `ConversationalChain` to track previous prompts and responses using the `ConversationalBufferMemory` class.

In [16]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

output_parser = CommaSeparatedListOutputParser()

conversation = ConversationChain(
    llm=llm,
    memory=ConversationBufferMemory()
)

conversation.predict(input="List all possible words as substitute for 'great' as comma separated.")

' Excellent, superb, wonderful, terrific, outstanding, remarkable, splendid, grand, fabulous, magnificent, glorious, sublime.'

In [17]:
conversation.predict(input="And the next 4?")

' Amazing, remarkable, incredible, and phenomenal.'

<hr>
<a class="anchor" id="seq">
    
### 1.4. Sequential Chain
    
</a>

Sequantial chain is desined to concatenate multiple chains into one. For example, the `SimpleSequentialChain` instance created below will start running each chain from the first index and pass its response to the next one in the list:

In [18]:
from langchain.chains import SimpleSequentialChain

#overall_chain = SimpleSequentialChain(chains=[chain_one, chain_two])

<hr>
<a class="anchor" id="debug">
    
### 1.5. Debug
    
</a>

To trace the inner work of a chain, one should set the `verbose` argument to `True` (if so, the output will depend on a specific application of chain).

In [19]:
template = """List all possible words as substitute for 'neural' as comma separated.

Current conversation:
{history}

{input}"""

conversation = ConversationChain(
    llm=llm,
    prompt=PromptTemplate(template=template, input_variables=["history", "input"], output_parser=output_parser),
    memory=ConversationBufferMemory(),
    verbose=True)

conversation.predict(input="")



> Entering new ConversationChain chain...
Prompt after formatting:
List all possible words as substitute for 'neural' as comma separated.

Current conversation:




> Finished chain.


'Brainy, Nervous, Nerve-wracking, Synaptic, Cognitive, Cerebral, Mental, Intellective, Thoughtful, Mindful, Psychogenic, Psychical, Psychosomatic.'

<hr>
<a class="anchor" id="custom">
    
### 1.6. Custom Chain
    
</a>


It is possible to define your own chain for any custom task:
1. Define a class that inherits most of its functionalities from the `Chain` class;
2. Declare three methods: `input_keys`, `output_keys` and `_call` (declation will depend on the specifics of the task).

In [20]:
# Create a custom chain (ConcatenateChain) that returns a word's meaning and then suggests a replacement
from langchain.chains import LLMChain
from langchain.chains.base import Chain
from typing import Dict, List


class ConcatenateChain(Chain):
    chain_1: LLMChain
    chain_2: LLMChain

    @property
    def input_keys(self) -> List[str]:
        # Union of the input keys of the two chains.
        all_input_vars = set(self.chain_1.input_keys).union(set(self.chain_2.input_keys))
        return list(all_input_vars)

    @property
    def output_keys(self) -> List[str]:
        return ['concat_output']

    def _call(self, inputs: Dict[str, str]) -> Dict[str, str]:
        output_1 = self.chain_1.run(inputs)
        output_2 = self.chain_2.run(inputs)
        return {'concat_output': output_1 + output_2}

In [21]:
# Declare each chain individually using the "LLMChain" class

# 1
prompt_1 = PromptTemplate(
    input_variables=["word"],
    template="What is the meaning of the following word '{word}'?",
)
chain_1 = LLMChain(llm=llm, prompt=prompt_1)

# 2
prompt_2 = PromptTemplate(
    input_variables=["word"],
    template="What is a word to replace the following: {word}?",
)
chain_2 = LLMChain(llm=llm, prompt=prompt_2)

In [22]:
# Call ConcatenateChain to merge the results of the chain_1 and chain_2
concat_chain = ConcatenateChain(chain_1=chain_1, chain_2=chain_2)
concat_output = concat_chain.run("network")

print(f"Concatenated output:\n{concat_output}")

Concatenated output:


Network is a term used to describe a system of interconnected components, such as computers, servers, and other devices, that are able to communicate with each other. It can also refer to the connections between people, such as social networks.

System


<hr>
<a class="anchor" id="video_summarizer">
    
## 2. YouTube Video Summarizer
    
</a>

<hr>
<a class="anchor" id="voice_assistant">
    
## 3. Creating Voice Assistant
    
</a>

<hr>
<a class="anchor" id="code_understanding">
    
## 4. Code Comprehension - Twitter Algorithm
    
</a>

<hr>
<a class="anchor" id="recommendation">
    
## 5. Recommendation Engine for Songs
    
</a>

<hr>
<a class="anchor" id="critique">
    
## 6. Self-Critique Chain
    
</a>

Self-critique chain acts as a mechanism to ensure model responses are appropriate in a production environment. By iterating over the model's output and checking against predefined expectations, the self-critique chain prompts the model to correct itself when necessary. 

<hr>
<a class="anchor" id="resources">
    
## 7. Additional Resources
    
</a>

- [Langchain documentation on Chains](https://python.langchain.com/docs/modules/chains/)